In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
load_data_stratified_kfold.py

• Removes the first 5 subjects of every class (per modality)
• Builds subject-wise stratified folds
• Balances **only the training fold** with SMOTE
• Preserves subject IDs (pid) for synthetic samples
• Scales features with MinMaxScaler (fit on train only)
• Returns: list[repetition] → list[fold] → (train_loader, val_loader)
"""

import os
import re
from pathlib import Path
from collections import Counter

import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from imblearn.over_sampling import SMOTE

# ────────────────────────────────────────────────────────────────
# Main utility
# ────────────────────────────────────────────────────────────────
def load_data_stratified_kfold(
    pte_directory: str,
    DE_directory: str,
    batch_size: int,
    selected_classes=("alz", "ctrl"),
    selected_channels=None,
    n_splits: int = 10,
    n_repetitions: int = 30,
    random_state: int = 42,
):
    # ────────── 1) Channel setup ──────────────────────────────
    ch_names = [
        "Fp1", "Fp2", "F3", "F4", "C3", "C4", "P3", "P4", "O1", "O2",
        "F7", "F8", "T3", "T4", "T5", "T6", "Fz", "Cz", "Pz",
    ]
    if selected_channels is None:
        selected_channels = ch_names
    sel_idx = [ch_names.index(ch) for ch in selected_channels]
    label_map = {c: i for i, c in enumerate(selected_classes)}

    # helper to parse filenames -> (subject_id, label)
    def parse_info(fname):
        m = re.match(r"sub-(\d+)_.*_(\w+)\.npz", fname)
        if not m:
            return None
        sid, lbl = int(m.group(1)), m.group(2).lower()
        if lbl not in selected_classes:
            return None
        return sid, lbl

    # ────────── 2) Collect file lists per modality ────────────
    def collect_files(directory):
        all_files = sorted(
            [f for f in os.listdir(directory) if f.endswith(".npz")],
            key=lambda f: int(re.search(r"sub-(\d+)_", f).group(1)),
        )
        info = [parse_info(f) + (f,) for f in all_files if parse_info(f) is not None]

        # drop the first 5 subjects per class
        drop_ids = {}
        for cls in selected_classes:
            ids = sorted({sid for sid, lbl, _ in info if lbl == cls})
            drop_ids[cls] = set(ids[:5])

        return [
            fname
            for sid, lbl, fname in info
            if sid not in drop_ids[lbl]
        ]

    pte_files = collect_files(pte_directory)
    psd_files = collect_files(DE_directory)

    # ────────── 3) Load arrays & build master tensors ─────────
    pte_list, psd_list, labels_list, pid_list = [], [], [], []

    for fname in pte_files:
        sid, lbl = parse_info(fname)
        lbl_int = label_map[lbl]
        arr = np.load(Path(pte_directory) / fname, allow_pickle=True)

        pte = arr["pte_data"]  # (N, 11, n_bands, n_ch, n_ch)
        pte = pte[:, :, :, sel_idx, :][:, :, :, :, sel_idx]

        N = pte.shape[0]
        pte_list.append(pte)
        labels_list.append(np.full(N, lbl_int, dtype=int))
        pid_list.extend([sid] * N)

    for fname in psd_files:
        sid, _ = parse_info(fname)
        arr = np.load(Path(DE_directory) / fname, allow_pickle=True)

        psd = arr["DE_features"]  # (N, n_ch, n_bands)
        psd = psd[:, sel_idx, :]

        psd_list.append(psd)

    X_pte = np.concatenate(pte_list, axis=0)
    X_psd = np.concatenate(psd_list, axis=0)
    y = np.concatenate(labels_list, axis=0)
    pid = np.asarray(pid_list, dtype=int)

    assert X_pte.shape[0] == X_psd.shape[0] == y.shape[0] == pid.shape[0]

    # ────────── 4) Subject labels for stratification ──────────
    unique_pids = np.unique(pid)
    subj_labels = np.array(
        [Counter(y[pid == sid]).most_common(1)[0][0] for sid in unique_pids]
    )

    # ────────── 5) Build repeated folds ───────────────────────
    all_reps = []
    for rep in range(n_repetitions):
        skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=rep)
        rep_folds = []

        for subj_tr_idx, subj_va_idx in skf.split(unique_pids, subj_labels):
            train_pids = unique_pids[subj_tr_idx]
            val_pids = unique_pids[subj_va_idx]

            tr_mask = np.isin(pid, train_pids)
            va_mask = np.isin(pid, val_pids)

            Xp_tr, Xp_va = X_pte[tr_mask], X_pte[va_mask]
            Xs_tr, Xs_va = X_psd[tr_mask], X_psd[va_mask]
            y_tr, y_va = y[tr_mask], y[va_mask]
            pid_tr, pid_va = pid[tr_mask], pid[va_mask]

            # ── flatten 
            flat_pte_tr = Xp_tr.reshape(len(y_tr), -1)
            flat_psd_tr = Xs_tr.reshape(len(y_tr), -1)
            X_train_flat = np.hstack([flat_pte_tr, flat_psd_tr])

            # ── SMOTE train-only 
            sm = SMOTE(random_state=rep)
            X_bal, y_bal = sm.fit_resample(X_train_flat, y_tr)

            # grab resampling indices (version-robust)
            if hasattr(sm, "sample_indices_"):
                res_idx = sm.sample_indices_
            elif hasattr(sm, "_sample_indices"):
                res_idx = sm._sample_indices
            else:
                # fallback: resample indices array
                idx = np.arange(len(y_tr)).reshape(-1, 1)
                idx_bal, _ = SMOTE(random_state=rep).fit_resample(idx, y_tr)
                res_idx = idx_bal.ravel()

            pid_bal = pid_tr[res_idx]

            # report synthetic count
            n_synth = len(X_bal) - len(X_train_flat)
            print(f"[rep {rep:02d}] SMOTE added {n_synth} synthetic samples")

            # ── split back to modalities
            split_at = flat_pte_tr.shape[1]
            flat_pte_bal = X_bal[:, :split_at]
            flat_psd_bal = X_bal[:, split_at:]

            # ── scale (fit on train only)
            scaler_pte = MinMaxScaler()
            scaler_psd = MinMaxScaler()

            flat_pte_bal = scaler_pte.fit_transform(flat_pte_bal)
            flat_pte_val = scaler_pte.transform(Xp_va.reshape(len(y_va), -1))

            flat_psd_bal = scaler_psd.fit_transform(flat_psd_bal)
            flat_psd_val = scaler_psd.transform(Xs_va.reshape(len(y_va), -1))

            # ── reshape back
            Xp_tr_bal = flat_pte_bal.reshape(-1, *Xp_tr.shape[1:])
            Xs_tr_bal = flat_psd_bal.reshape(-1, *Xs_tr.shape[1:])
            Xp_va = flat_pte_val.reshape(Xp_va.shape)
            Xs_va = flat_psd_val.reshape(Xs_va.shape)

            # ── DataLoaders
            def make_loader(x1, x2, y_, p_, shuffle):
                t1 = torch.from_numpy(x1).float()
                t2 = torch.from_numpy(x2).float()
                ty = torch.from_numpy(y_).long()
                tp = torch.from_numpy(p_).long()
                ds = TensorDataset(t1, t2, ty, tp)
                return DataLoader(ds, batch_size=batch_size, shuffle=shuffle, drop_last=False)

            train_loader = make_loader(Xp_tr_bal, Xs_tr_bal, y_bal, pid_bal, shuffle=True)
            val_loader   = make_loader(Xp_va,      Xs_va,    y_va, pid_va,  shuffle=False)

            rep_folds.append((train_loader, val_loader))

        all_reps.append(rep_folds)

    return all_reps


# ────────────────────────────────────────────────────────────────
# Demo / quick smoke-test
# ────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    pte_dir   = "features"
    de_dir    = "DE_features_single_window"
    bs        = 32
    sel_chans = ["Fp1", "Fp2", "F3", "F4"]
    n_folds   = 15

    folds_by_rep = load_data_stratified_kfold(
        pte_directory=pte_dir,
        DE_directory=de_dir,
        batch_size=bs,
        selected_classes=("alz", "ctrl"),
        selected_channels=sel_chans,
        n_splits=n_folds,
        n_repetitions=1,     # keep small for the smoke-test
    )

    # print a quick sanity check of the first repetition
    rep0 = folds_by_rep[0]
    for k, (tr, va) in enumerate(rep0, 1):
        print(f"\nFold {k:02d}: batches → train {len(tr)}, val {len(va)}")
        xpte, xpsd, y, p = next(iter(tr))
        print("   Train batch shapes :", xpte.shape, xpsd.shape, y.shape, p.shape)
        xpte, xpsd, y, p = next(iter(va))
        print("   Valid batch shapes :", xpte.shape, xpsd.shape, y.shape, p.shape)


In [ ]:
import math
import torch
from itertools import cycle
import torch.nn.functional as F


def train_model(
    model,
    source_dataloader,
    target_dataloader,
    criterion_label,
    criterion_domain,
    optimizer,
    num_epochs=10,
    device="cuda",
    alpha_entropy = 0.01,
    scheduler=None,
):
   
    model.to(device)
    model.train()

    accuracy_history = []
    domain_accuracy_history = []

    for epoch in range(num_epochs):
        total_correct = 0
        total_samples = 0
        total_domain_correct = 0
        total_domain_samples = 0

        epoch_loss = 0.0


        for i, batch_src in enumerate(source_dataloader):

            if len(batch_src) == 4:
                source_pte, source_psd, source_labels, _ = batch_src
            else:
                source_pte, source_psd, source_labels, _ = batch_src


            # Move data to device
            source_pte = source_pte.to(device)
            source_psd = source_psd.to(device)
            source_labels = source_labels.to(device)


            label_preds, _ = model(
                source_pte, source_psd
            )
 


            loss_label = criterion_label(label_preds, source_labels)


            total_loss = loss_label
            # Backward + Optimize
            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()

            # -------------------------------
            # Update Metrics
            # -------------------------------
            epoch_loss += loss_label.item()

            # Label prediction accuracy (source)
            _, predicted = torch.max(label_preds, dim=1)
            correct = (predicted == source_labels).sum().item()
            total_correct += correct
            total_samples += source_labels.size(0)

        if scheduler is not None:
            scheduler.step()

            
        epoch_accuracy = 100.0 * total_correct / total_samples if total_samples > 0 else 0
        epoch_domain_accuracy = 100.0 * total_domain_correct / total_domain_samples if total_domain_samples > 0 else 0

        accuracy_history.append(epoch_accuracy)
        domain_accuracy_history.append(epoch_domain_accuracy)

        # print(f"Epoch [{epoch+1}/{num_epochs}] "
        #       f"Label Acc: {epoch_accuracy:.2f}%, "
        #       f"Domain Acc: {epoch_domain_accuracy:.2f}%")

    return accuracy_history, domain_accuracy_history


In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
from typing import Tuple
from sklearn.metrics import f1_score

def test_model(
    model: torch.nn.Module,
    test_dataloader: torch.utils.data.DataLoader,
    criterion_label: torch.nn.Module,
    device: str = "cuda",
    num_classes: int = 2,
    alz_threshold: float = 0.4
) -> Tuple[
    float,        # avg sample loss
    float,        # mean participant accuracy (%)
    float,        # mean participant F1 (macro)
    np.ndarray,   # summed confusion matrix (num_classes×num_classes)
    np.ndarray,   # all sample softmax probs, shape (N, num_classes)
    np.ndarray,   # all sample true labels,   shape (N,)
    np.ndarray,   # per‐participant alz_ratio, shape (P,)
    np.ndarray    # per‐participant true labels, shape (P,)
]:
    model.to(device).eval()
    total_loss = 0.0

    all_preds  = []
    all_labels = []
    all_probs  = []
    all_pids   = []

    with torch.no_grad():
        for batch in test_dataloader:
            # unpack 3 or 4 items
            if len(batch) == 4:
                pte, psd, labels, pids = batch
            else:
                pte, psd, labels = batch
                pids = torch.zeros_like(labels)

            pte, psd, labels = pte.to(device), psd.to(device), labels.to(device)
            logits, _ = model(pte, psd)
            loss = criterion_label(logits, labels)
            total_loss += loss.item()

            probs = F.softmax(logits, dim=1)
            preds = probs.argmax(dim=1)

            all_probs .append(probs.cpu().numpy())
            all_preds .extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_pids  .extend(pids.cpu().numpy())

    # average sample loss
    n_batches = len(test_dataloader)
    avg_loss  = total_loss / n_batches if n_batches else 0.0

    # flatten lists → arrays
    all_probs  = np.vstack(all_probs)        # (N, num_classes)
    all_preds  = np.array(all_preds)         # (N,)
    all_labels = np.array(all_labels)        # (N,)
    all_pids   = np.array(all_pids)          # (N,)

    # group by participant
    part_ids    = np.unique(all_pids)
    part_accs   = []
    part_preds  = []
    part_confs  = np.zeros((num_classes, num_classes), dtype=int)
    part_ratios = []
    part_trues  = []

    for pid in part_ids:
        mask = (all_pids == pid)
        labs = all_labels[mask]
        preds= all_preds[mask]

        # assume each participant’s true label is constant
        true_lbl = labs[0]

        # fraction of windows predicted as "alz"
        alz_ratio = (preds == 1).sum() / max(len(preds), 1)
        pred_lbl  = 1 if alz_ratio >= alz_threshold else 0

        part_confs[true_lbl, pred_lbl] += 1
        part_accs.append(100.0 if pred_lbl == true_lbl else 0.0)

        part_preds.append(pred_lbl)
        part_ratios.append(alz_ratio)
        part_trues.append(true_lbl)

    mean_acc = float(np.mean(part_accs)) if part_accs else 0.0
    # compute true participant-level macro-F1
    mean_f1 = (f1_score(part_trues, part_preds,
                        average='macro', zero_division=0)
               if part_trues else 0.0)

    return (
        avg_loss,                    # sample‐level loss
        mean_acc,                    # mean participant accuracy
        mean_f1,                     # mean participant F1 (macro)
        part_confs,                  # summed conf matrix
        all_probs,                   # sample‐level probs
        all_labels,                  # sample‐level labels
        np.array(part_ratios),       # per‐participant alz ratios
        np.array(part_trues)         # per‐participant true labels
    )


In [ ]:
import torch
import torch.nn.functional as F
from collections import defaultdict
from sklearn.metrics import f1_score, accuracy_score

def tune_threshold_on_source(
    model,
    source_dataloader,
    device="cuda",
    thresholds=[0.1, 0.2, 0.3, 0.4, 0.5],
    num_classes=2
):

    model.eval()
    model.to(device)
    sample_preds = defaultdict(list)
    participant_label = {}

    with torch.no_grad():
        for batch in source_dataloader:
            # If your source dataloader returns 4 items, including participant_id
            if len(batch) == 4:
                pte_batch, psd_batch, labels, pid_batch = batch
            else:
                raise ValueError("Expected Dataloader to return (pte, psd, labels, pid).")

            pte_batch = pte_batch.to(device)
            psd_batch = psd_batch.to(device)
            labels = labels.to(device)
            pid_batch = pid_batch.to(device)

            # Forward pass (disable GRL in inference by setting lambda_=0)
            label_preds, _ = model(pte_batch, psd_batch)

            # Convert predictions to class=0/1
            softmax_output = F.softmax(label_preds, dim=1)
            _, predicted = torch.max(softmax_output, dim=1)

            # Move to CPU
            predicted = predicted.cpu().numpy()
            labels = labels.cpu().numpy()
            pid_batch = pid_batch.cpu().numpy()

            # Store predictions for each participant
            for pred, true_lbl, pid in zip(predicted, labels, pid_batch):
                sample_preds[pid].append(pred)
                # We assume all samples from participant pid share the same ground-truth label:
                if pid not in participant_label:
                    participant_label[pid] = true_lbl
                else:
                    # Optionally check that the label is consistent
                    if participant_label[pid] != true_lbl:
                        raise ValueError(f"Inconsistent labels for participant {pid} in source data.")

    # Now we have sample-level predictions per participant. We'll try each threshold.
    best_threshold = None
    best_metric_val = -1.0

    for thr in thresholds:
        # For each threshold, generate participant-level predictions
        # by counting how many samples predicted as class=1
        part_level_preds = []
        part_level_trues = []

        for pid, preds_list in sample_preds.items():
            true_lbl = participant_label[pid]
            n_alz = sum([p == 1 for p in preds_list])
            ratio = float(n_alz) / len(preds_list)
            # Decide participant-level label
            if ratio >= thr:
                participant_pred = 1
            else:
                participant_pred = 0
            
            part_level_preds.append(participant_pred)
            part_level_trues.append(true_lbl)

        # Evaluate participant-level performance
        # For example, we use F1 (macro):
        f1 = f1_score(part_level_trues, part_level_preds, average='macro', zero_division=0)
        acc = accuracy_score(part_level_trues, part_level_preds)

        print(f"[Threshold {thr}] -> F1={f1:.4f} | Acc={acc:.4f}")

        # Suppose we pick the threshold that maximizes participant-level F1
        if f1 > best_metric_val:
            best_metric_val = f1
            best_threshold = thr

    print(f"\n[Best Threshold] = {best_threshold} with F1={best_metric_val:.4f}")
    return best_threshold


In [ ]:
import torch.nn as nn
import torch
import torch.nn.functional as F
import math


# Define the Multi-Head Cross Attention Module
class MultiHeadCrossAttention(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.1):

        super(MultiHeadCrossAttention, self).__init__()
        self.multihead_attn = nn.MultiheadAttention(embed_dim=d_model, num_heads=num_heads, dropout=dropout, batch_first=True)
        self.layer_norm = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, query, key, value, attn_mask=None, key_padding_mask=None):

        # Apply MultiheadAttention; note that nn.MultiheadAttention expects inputs of shape (batch, seq, feature)
        attn_output, attn_weights = self.multihead_attn(query, key, value, attn_mask=attn_mask, key_padding_mask=key_padding_mask)
        
        # Apply dropout
        attn_output = self.dropout(attn_output)
        
        # Add & Norm
        output = self.layer_norm(query + attn_output)
        
        return output, attn_weights

# Existing Transformer Classes (PteTransformer and PsdTransformer)
class PteTransformer(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_heads, output_dim, dropout):
        super(PteTransformer, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.num_heads = num_heads
        self.output_dim = output_dim
        self.dropout = dropout
        self.position_encoding = nn.Parameter(torch.randn(1, 11, input_dim), requires_grad=True)

        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim,
            nhead=num_heads,
            dim_feedforward=hidden_dim,
            dropout=dropout,
            batch_first=True,
            activation="gelu"
        )
        self.transformer = nn.TransformerEncoder(encoder_layer=self.encoder_layer, num_layers=num_layers)
        self.output_layer = nn.Linear(input_dim, output_dim)  # Project to desired output_dim

    def forward(self, x):
        """
        x: (batch_size, original_features)
        """
        b = x.shape[0]
        win = x.shape[1]
        # Reshape to (batch_size, 11, input_dim//11) assuming input_dim is divisible by 11
        x = x.reshape(b, 11, -1)
        x = self.position_encoding + x  # (batch_size, 11, input_dim)
        x = self.transformer(x)         # (batch_size, 11, input_dim)
        x = self.output_layer(x)        # (batch_size, 11, output_dim)
        return x

class PsdTransformer(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_heads, output_dim, dropout):
        super(PsdTransformer, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.num_heads = num_heads
        self.output_dim = output_dim
        self.dropout = dropout

        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim,
            nhead=num_heads,
            dim_feedforward=hidden_dim,
            dropout=dropout,
            batch_first=True,
            activation="gelu"
        )
        self.transformer = nn.TransformerEncoder(encoder_layer=self.encoder_layer, num_layers=num_layers)
        self.output_layer = nn.Linear(input_dim, output_dim)  # Project to desired output_dim

    def forward(self, x):
        """
        x: (batch_size, original_features)
        """
        # Assuming x is already of shape (batch_size, T_e, input_dim)
        x = self.transformer(x)         # (batch_size, T_e, input_dim)
        x = self.output_layer(x)        # (batch_size, T_e, output_dim)
        return x

# Updated Final Model with Multi-Head Cross Attention and DANN Components
class FinalModel(nn.Module):
    def __init__(self, 
                 pte_input_dim, pte_hidden_dim, pte_num_layers, pte_num_heads, pte_output_dim, pte_dropout,
                 psd_input_dim, psd_hidden_dim, psd_num_layers, psd_num_heads, psd_output_dim, psd_dropout,
                 cross_d_model, cross_num_heads,
                 ):
        super(FinalModel, self).__init__() 

        # Initialize PTE and PSD Transformers
        self.pte_transformer = PteTransformer(
            input_dim=pte_input_dim,
            hidden_dim=pte_hidden_dim,
            num_layers=pte_num_layers,
            num_heads=pte_num_heads,
            output_dim=pte_output_dim,
            dropout=pte_dropout
        )
        
        self.psd_transformer = PsdTransformer(
            input_dim=psd_input_dim,
            hidden_dim=psd_hidden_dim,
            num_layers=psd_num_layers,
            num_heads=psd_num_heads,
            output_dim=psd_output_dim,
            dropout=psd_dropout
        )
        
        # Initialize Multi-Head Cross-Attention
        self.cross_attention = MultiHeadCrossAttention(
            d_model=cross_d_model,
            num_heads=cross_num_heads,
            dropout=psd_dropout
        )

        # Final Classifier for Label Prediction
        self.final_classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.7),
            nn.Linear(1408, 2)  # Adjust based on the sequence length (12 here)
        )
        
    def forward(self, pte_input, psd_input):

        # Pass through respective transformers
        pte_encoded = self.pte_transformer(pte_input)  # (batch_size, T_pte=11, pte_output_dim=128)
        psd_encoded = self.psd_transformer(psd_input)  # (batch_size, T_psd=6, psd_output_dim=128)
        
        # print(psd_encoded.shape)
        # Apply multi-head cross-attention: Let pte_encoded attend to psd_encoded
        cross_attn_output, attn_weights = self.cross_attention(
            query=pte_encoded,      # (batch_size, T_q=11, d_model=128)
            key=psd_encoded,        # (batch_size, T_k=6, d_model=128)
            value=psd_encoded       # (batch_size, T_k=6, d_model=128)
        )
        
        # Label Prediction
        label_pred = self.final_classifier(cross_attn_output)  # (batch_size, 2)
        
        
        return label_pred, attn_weights

# Example usage
if __name__ == "__main__":
    # Example input tensors
    # Adjust the shapes based on your actual data
    # For illustration, assuming:
    # pte_input has 11 time steps, each with (input_dim_pte / 11) features
    x_1 = torch.randn(32, 11 * 5 * 6 * 6)  # (batch_size, pte_original_features)
    # psd_input has sequence length 6 with 5 features each
    x_2 = torch.randn(32, 6, 5)            # (batch_size, psd_seq_length, psd_original_features)
    
    # Define the model parameters
    input_dim_pte = 180   # Example: 11 * 5 * 6 * 6 = 180
    hidden_dim_pte = 512
    num_layers_pte = 2
    num_heads_pte = 5     # Typically, num_heads should divide d_model
    output_dim_pte = 128
    dropout_pte = 0.4
    
    input_dim_psd = 5
    hidden_dim_psd = 512
    num_layers_psd = 2
    num_heads_psd = 5    # Typically, num_heads should divide d_model
    output_dim_psd = 128
    dropout_psd = 0.4
    
    cross_d_model = 128
    cross_num_heads = 8   # Number of heads in cross-attention
    
    # Initialize the DANN model
    model = FinalModel(
        pte_input_dim=input_dim_pte, 
        pte_hidden_dim=hidden_dim_pte, 
        pte_num_layers=num_layers_pte, 
        pte_num_heads=num_heads_pte, 
        pte_output_dim=output_dim_pte, 
        pte_dropout=dropout_pte,
        psd_input_dim=input_dim_psd, 
        psd_hidden_dim=hidden_dim_psd, 
        psd_num_layers=num_layers_psd, 
        psd_num_heads=num_heads_psd, 
        psd_output_dim=output_dim_psd, 
        psd_dropout=dropout_psd,
        cross_d_model=cross_d_model, 
        cross_num_heads=cross_num_heads
    )
    
    # Example forward pass
    label_pred, attn_weights = model(x_1, x_2)
    print("Label Prediction Shape:", label_pred.shape)        # Expected: (32, 2)
    print("Attention Weights Shape:", attn_weights.shape)
    
# summary(model, input_size=[(32, 11 * 5 * 6 * 6), (32, 6, 5)], col_names=["input_size", "output_size", "num_params", "trainable"], col_width=20)
from torchinfo import summary

summary(model=model,
        input_size=[(32, 11, 5 * 6 * 6), (32, 6, 5)], # (batch_size, color_channels, height, width)
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)


In [ ]:
import random
import numpy as np
import torch
from sklearn.utils import check_random_state

def set_seed(seed):
    random.seed(seed)

    # Set the seed for numpy
    np.random.seed(seed)

    # Set the seed for PyTorch (both CPU and CUDA)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # For multi-GPU setups

    # Ensure deterministic behavior in PyTorch (if applicable)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # Set the seed for sklearn (via check_random_state)
    _ = check_random_state(seed)

    print(f"Seed set to: {seed}")


In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score, roc_curve
from statistics import mode, StatisticsError
import matplotlib.pyplot as plt

seeds = [0, 1, 2 ,3, 4 , 5 , 6 ,7, 8, 9]  # Example seeds for reproducibility

all_acc_final   = []
all_f1_final    = []
all_conf_final  = []
global_probs_final  = []
global_labels_final = []
best_thresholds_final = []

set_seed(0)
#for seed in seeds:
#    set_seed(seed)  # reproducibility

# model hyperparameters (unchanged)
input_dim_pte  = 180
hidden_dim_pte = 512
num_layers_pte = 2
num_heads_pte  = 5
output_dim_pte = 128
dropout_pte    = 0.4

input_dim_psd  = 5
hidden_dim_psd = 512
num_layers_psd = 2
num_heads_psd  = 5
output_dim_psd = 128
dropout_psd    = 0.4

cross_d_model   = 128
cross_num_heads = 8



# get stratified folds
all_folds  = load_data_stratified_kfold(
    pte_directory="features",
    DE_directory="DE_features_single_window",
    batch_size=32,
    selected_classes=["ctrl","alz"],
    selected_channels=['O1','O2','T4','T5','F7','F8'],
    n_splits=10,
    n_repetitions=30,
)


for folds in all_folds:

    # accumulators
    all_acc   = []
    all_f1    = []
    all_conf  = []
    global_probs  = []
    global_labels = []
    best_thresholds = []


    for fold_idx, (train_loader, val_loader) in enumerate(folds, 1):
        print(f"\n===== Fold {fold_idx}/{len(folds)} =====")

        # 1) model init
        model = FinalModel(
            pte_input_dim=input_dim_pte,
            pte_hidden_dim=hidden_dim_pte,
            pte_num_layers=num_layers_pte,
            pte_num_heads=num_heads_pte,
            pte_output_dim=output_dim_pte,
            pte_dropout=dropout_pte,
            psd_input_dim=input_dim_psd,
            psd_hidden_dim=hidden_dim_psd,
            psd_num_layers=num_layers_psd,
            psd_num_heads=num_heads_psd,
            psd_output_dim=output_dim_psd,
            psd_dropout=dropout_psd,
            cross_d_model=cross_d_model,
            cross_num_heads=cross_num_heads
        )

        device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
        model.to(device)

        # 2) loss & optimizer
        # class_weights = torch.tensor([1.0, 0.8], device=device)
        #balance the classes weights
        # from sklearn.utils.class_weight import compute_class_weight
        # labels = []
        # for _, _, y, _ in train_loader:
        #     labels.extend(y.numpy())
        # labels = np.array(labels)
        # class_weights = compute_class_weight(
        #     class_weight='balanced',
        #     classes=np.unique(labels),
        #     y=labels
        # )
        # class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)
        class_weights = torch.tensor([1.0, 0.7], device=device)
        # print(f"Class weights: {class_weights}")
        criterion_label = nn.CrossEntropyLoss(class_weights)
        # criterion_label = FocalLoss(gamma=2, alpha=class_weights, reduction='mean', task_type='multi-class', num_classes=2)

        optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
        # scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
        # 3) train
        num_epochs = 100
        label_acc_history, _ = train_model(
            model=model,
            source_dataloader=train_loader,
            target_dataloader=val_loader,
            criterion_label=criterion_label,
            criterion_domain=criterion_label,
            optimizer=optimizer,
            num_epochs=num_epochs,
            device=device,
            scheduler=None,
        )
        print(f"Final training label acc: {label_acc_history[-1]:.2f}%")

        # 4) threshold tuning on train set
        thresholds_to_try = [0.2,0.3,0.4,0.5]
        best_thr = tune_threshold_on_source(
            model=model,
            source_dataloader=train_loader,
            device=device,
            thresholds=thresholds_to_try,
            num_classes=2
        )
        best_thresholds.append(best_thr)

        # 5) test on validation set
        test_loss, test_acc, test_f1, conf_mat, preds, labels, alz_ratio, true_label = test_model(
            model=model,
            test_dataloader=val_loader,
            criterion_label=criterion_label,
            device=device,
            num_classes=2,
            alz_threshold=best_thr
        )

        print(f"Final validation loss: {test_loss:.4f}")
        print(f"Final validation acc: {test_acc:.2f}%")
        print(f"Final validation f1: {test_f1:.2f}%")

        # 6) record metrics
        all_acc.append(test_acc)
        all_f1.append(test_f1)
        all_conf.append(conf_mat)
        global_probs.append(preds)
        global_labels.append(labels)
        best_thresholds.append(best_thr)

    # ────────── 7) Final results ─────────────────────────────
    all_acc_final.append(all_acc)
    all_f1_final.append(all_f1)
    all_conf_final.append(all_conf)
    global_probs_final.append(global_probs)
    global_labels_final.append(global_labels)
    best_thresholds_final.append(best_thresholds)

    print(f"\n=== Cross‐Val Results ===")
    print(f"Accuracies per fold: {all_acc}")
    print(f"Mean acc: {np.mean(all_acc):.2f}%, Std: {np.std(all_acc):.2f}%")
    print(f"Mean f1: {np.mean(all_f1):.2f}%, Std: {np.std(all_f1):.2f}%")

final_results = {
"all_acc": all_acc_final,
"all_f1": all_f1_final,
"all_conf": all_conf_final,
"global_probs": global_probs_final,
"global_labels": global_labels_final,
"best_thresholds": best_thresholds_final
}

np.savez(
"final_results_cn_ad_dtca.npz",
final_results=final_results
)


In [ ]:
accuarcy = final_results["all_acc"]
f1 = final_results["all_f1"]
confusion_matrix = final_results["all_conf"]
global_probs = final_results["global_probs"]
global_labels = final_results["global_labels"]
best_thresholds = final_results["best_thresholds"]

import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# final_results["all_conf"] is a list of 10 runs, each a (5,2,2) array of confusion matrices
all_runs = final_results["all_conf"]

acc_scores, precision_scores, recall_scores, f1_scores = [], [], [], []

for run_idx, run_cms in enumerate(all_runs, start=1):
    # run_cms has shape (5,2,2): 5 folds × 2 × 2
    for fold_idx, cm in enumerate(run_cms, start=1):
        cm = np.asarray(cm)
        if cm.shape != (2,2):
            raise ValueError(f"Run {run_idx} Fold {fold_idx} has shape {cm.shape}, expected (2,2)")

        tn, fp, fn, tp = cm.ravel()

        # Reconstruct the true/pred arrays
        y_true = np.array([0]*(tn+fp) + [1]*(fn+tp))
        y_pred = np.array([0]*tn     + [1]*fp    +
                          [0]*fn     + [1]*tp)

        # Compute metrics
        acc_scores.append(   accuracy_score(y_true, y_pred))
        precision_scores.append(precision_score(y_true, y_pred, zero_division=0))
        recall_scores.append(   recall_score(y_true, y_pred, zero_division=0))
        f1_scores.append(       f1_score(y_true, y_pred,average="macro", zero_division=0))

# Now aggregate across all 50 confusion matrices
metrics = {
    "Accuracy":  (np.mean(acc_scores),       np.std(acc_scores)),
    "Precision": (np.mean(precision_scores), np.std(precision_scores)),
    "Recall":    (np.mean(recall_scores),    np.std(recall_scores)),
    "F1-score":  (np.mean(f1_scores),        np.std(f1_scores)),
}

for name, (mean, std) in metrics.items():
    print(f"{name}: Mean = {mean:.4f}, Std = {std:.4f}")



In [ ]:
# global_probs = np.hstack(global_probs)

gp = []
for i in range(len(global_probs)):
    gp.extend(global_probs[i])
global_probs = np.vstack(gp)
print(global_probs.shape)

gl = []
for i in range(len(global_labels)):
    gl.extend(global_labels[i])
global_labels = np.hstack(gl)
print(global_labels.shape)
print(global_labels)


In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

# Compute global sample-level AUC
global_auc = roc_auc_score(global_labels, global_probs[:, 1])
print(f"\nGlobal Sample-Level AUC: {global_auc:.4f}")

#plot ROC curve
fpr, tpr, _ = roc_curve(global_labels, global_probs[:, 1])
plt.figure(figsize=(6, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f"Global AUC = {global_auc:.4f}")
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Global ROC Curve')
plt.legend(loc="lower right")
plt.grid()
plt.show()



In [10]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score, roc_curve
from statistics import mode, StatisticsError
import matplotlib.pyplot as plt

seeds = [0, 1, 2 ,3, 4 , 5 , 6 ,7, 8, 9]  # Example seeds for reproducibility

all_acc_final   = []
all_f1_final    = []
all_conf_final  = []
global_probs_final  = []
global_labels_final = []
best_thresholds_final = []

set_seed(0)
#for seed in seeds:
#    set_seed(seed)  # reproducibility

# model hyperparameters (unchanged)
input_dim_pte  = 180
hidden_dim_pte = 512
num_layers_pte = 2
num_heads_pte  = 5
output_dim_pte = 128
dropout_pte    = 0.4

input_dim_psd  = 5
hidden_dim_psd = 512
num_layers_psd = 2
num_heads_psd  = 5
output_dim_psd = 128
dropout_psd    = 0.4

cross_d_model   = 128
cross_num_heads = 8



# get stratified folds
all_folds  = load_data_stratified_kfold(
    pte_directory="features",
    DE_directory="DE_features_single_window",
    batch_size=32,
    selected_classes=["ctrl","ftd"],
    selected_channels=['O1','O2','T4','T5','F7','F8'],
    n_splits=10,
    n_repetitions=30,
)


for folds in all_folds:

    # accumulators
    all_acc   = []
    all_f1    = []
    all_conf  = []
    global_probs  = []
    global_labels = []
    best_thresholds = []


    for fold_idx, (train_loader, val_loader) in enumerate(folds, 1):
        print(f"\n===== Fold {fold_idx}/{len(folds)} =====")

        # 1) model init
        model = FinalModel(
            pte_input_dim=input_dim_pte,
            pte_hidden_dim=hidden_dim_pte,
            pte_num_layers=num_layers_pte,
            pte_num_heads=num_heads_pte,
            pte_output_dim=output_dim_pte,
            pte_dropout=dropout_pte,
            psd_input_dim=input_dim_psd,
            psd_hidden_dim=hidden_dim_psd,
            psd_num_layers=num_layers_psd,
            psd_num_heads=num_heads_psd,
            psd_output_dim=output_dim_psd,
            psd_dropout=dropout_psd,
            cross_d_model=cross_d_model,
            cross_num_heads=cross_num_heads
        )

        device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
        model.to(device)

        # 2) loss & optimizer
        # class_weights = torch.tensor([1.0, 0.8], device=device)
        #balance the classes weights
        # from sklearn.utils.class_weight import compute_class_weight
        # labels = []
        # for _, _, y, _ in train_loader:
        #     labels.extend(y.numpy())
        # labels = np.array(labels)
        # class_weights = compute_class_weight(
        #     class_weight='balanced',
        #     classes=np.unique(labels),
        #     y=labels
        # )
        # class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)
        # class_weights = torch.tensor([1.0, 0.7], device=device)
        # print(f"Class weights: {class_weights}")
        criterion_label = nn.CrossEntropyLoss()
        # criterion_label = FocalLoss(gamma=2, alpha=class_weights, reduction='mean', task_type='multi-class', num_classes=2)

        optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
        # scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
        # 3) train
        num_epochs = 100
        label_acc_history, _ = train_model(
            model=model,
            source_dataloader=train_loader,
            target_dataloader=val_loader,
            criterion_label=criterion_label,
            criterion_domain=criterion_label,
            optimizer=optimizer,
            num_epochs=num_epochs,
            device=device,
            scheduler=None,
        )
        print(f"Final training label acc: {label_acc_history[-1]:.2f}%")

        # 4) threshold tuning on train set
        thresholds_to_try = [0.2,0.3,0.4,0.5]
        best_thr = tune_threshold_on_source(
            model=model,
            source_dataloader=train_loader,
            device=device,
            thresholds=thresholds_to_try,
            num_classes=2
        )
        best_thresholds.append(best_thr)

        # 5) test on validation set
        test_loss, test_acc, test_f1, conf_mat, preds, labels, alz_ratio, true_label = test_model(
            model=model,
            test_dataloader=val_loader,
            criterion_label=criterion_label,
            device=device,
            num_classes=2,
            alz_threshold=best_thr
        )

        print(f"Final validation loss: {test_loss:.4f}")
        print(f"Final validation acc: {test_acc:.2f}%")
        print(f"Final validation f1: {test_f1:.2f}%")

        # 6) record metrics
        all_acc.append(test_acc)
        all_f1.append(test_f1)
        all_conf.append(conf_mat)
        global_probs.append(preds)
        global_labels.append(labels)
        best_thresholds.append(best_thr)

    # ────────── 7) Final results ─────────────────────────────
    all_acc_final.append(all_acc)
    all_f1_final.append(all_f1)
    all_conf_final.append(all_conf)
    global_probs_final.append(global_probs)
    global_labels_final.append(global_labels)
    best_thresholds_final.append(best_thresholds)

    print(f"\n=== Cross‐Val Results ===")
    print(f"Accuracies per fold: {all_acc}")
    print(f"Mean acc: {np.mean(all_acc):.2f}%, Std: {np.std(all_acc):.2f}%")
    print(f"Mean f1: {np.mean(all_f1):.2f}%, Std: {np.std(all_f1):.2f}%")

final_results = {
"all_acc": all_acc_final,
"all_f1": all_f1_final,
"all_conf": all_conf_final,
"global_probs": global_probs_final,
"global_labels": global_labels_final,
"best_thresholds": best_thresholds_final
}

np.savez(
    "final_results_cn_ftd_dtca_no_weights.npz",
    final_results=final_results
)

Seed set to: 0
[rep 00] SMOTE added 87 synthetic samples
[rep 00] SMOTE added 83 synthetic samples
[rep 00] SMOTE added 71 synthetic samples
[rep 00] SMOTE added 71 synthetic samples
[rep 00] SMOTE added 91 synthetic samples
[rep 00] SMOTE added 90 synthetic samples
[rep 00] SMOTE added 103 synthetic samples
[rep 00] SMOTE added 102 synthetic samples
[rep 00] SMOTE added 95 synthetic samples
[rep 00] SMOTE added 98 synthetic samples
[rep 01] SMOTE added 88 synthetic samples
[rep 01] SMOTE added 82 synthetic samples
[rep 01] SMOTE added 70 synthetic samples
[rep 01] SMOTE added 67 synthetic samples
[rep 01] SMOTE added 94 synthetic samples
[rep 01] SMOTE added 97 synthetic samples
[rep 01] SMOTE added 102 synthetic samples
[rep 01] SMOTE added 99 synthetic samples
[rep 01] SMOTE added 96 synthetic samples
[rep 01] SMOTE added 96 synthetic samples
[rep 02] SMOTE added 86 synthetic samples
[rep 02] SMOTE added 83 synthetic samples
[rep 02] SMOTE added 68 synthetic samples
[rep 02] SMOTE a

/home/s.dharia-ra/miniconda3/envs/ml/lib/python3.10/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(


Final training label acc: 83.45%
[Threshold 0.2] -> F1=0.8103 | Acc=0.8108
[Threshold 0.3] -> F1=0.8377 | Acc=0.8378
[Threshold 0.4] -> F1=0.8645 | Acc=0.8649
[Threshold 0.5] -> F1=0.8912 | Acc=0.8919

[Best Threshold] = 0.5 with F1=0.8912
Final validation loss: 0.8513
Final validation acc: 80.00%
Final validation f1: 0.80%

===== Fold 2/10 =====


/home/s.dharia-ra/miniconda3/envs/ml/lib/python3.10/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(


Final training label acc: 83.87%
[Threshold 0.2] -> F1=0.7539 | Acc=0.7568
[Threshold 0.3] -> F1=0.8377 | Acc=0.8378
[Threshold 0.4] -> F1=0.8645 | Acc=0.8649
[Threshold 0.5] -> F1=0.9180 | Acc=0.9189

[Best Threshold] = 0.5 with F1=0.9180
Final validation loss: 1.3248
Final validation acc: 60.00%
Final validation f1: 0.58%

===== Fold 3/10 =====


...

Final training label acc: 80.07%
[Threshold 0.2] -> F1=0.8157 | Acc=0.8158
[Threshold 0.3] -> F1=0.8417 | Acc=0.8421
[Threshold 0.4] -> F1=0.8936 | Acc=0.8947
[Threshold 0.5] -> F1=0.9183 | Acc=0.9211

[Best Threshold] = 0.5 with F1=0.9183
Final validation loss: 0.5396
Final validation acc: 100.00%
Final validation f1: 1.00%

===== Fold 9/10 =====


/home/s.dharia-ra/miniconda3/envs/ml/lib/python3.10/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(


Final training label acc: 79.04%
[Threshold 0.2] -> F1=0.8157 | Acc=0.8158
[Threshold 0.3] -> F1=0.8676 | Acc=0.8684
[Threshold 0.4] -> F1=0.9460 | Acc=0.9474
[Threshold 0.5] -> F1=0.9183 | Acc=0.9211

[Best Threshold] = 0.4 with F1=0.9460
Final validation loss: 0.2555
Final validation acc: 100.00%
Final validation f1: 1.00%

===== Fold 10/10 =====


/home/s.dharia-ra/miniconda3/envs/ml/lib/python3.10/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(


Final training label acc: 79.07%
[Threshold 0.2] -> F1=0.8676 | Acc=0.8684
[Threshold 0.3] -> F1=0.8936 | Acc=0.8947
[Threshold 0.4] -> F1=0.9449 | Acc=0.9474
[Threshold 0.5] -> F1=0.9164 | Acc=0.9211

[Best Threshold] = 0.4 with F1=0.9449
Final validation loss: 0.8054
Final validation acc: 50.00%
Final validation f1: 0.33%

=== Cross‐Val Results ===
Accuracies per fold: [40.0, 80.0, 0.0, 100.0, 50.0, 75.0, 50.0, 100.0, 100.0, 50.0]
Mean acc: 64.50%, Std: 30.86%
Mean f1: 0.61%, Std: 0.33%
